# Toronto Neighborhoods

Task: Scrape the following Wikipedia page into a dataframe: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 
    
       The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
       Ignore cells with a borough that is Not assigned
       More than one neighborhood can exist in one postal code area - combine them into one row with the neighborhoods separated with a comma
       If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
       In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
       Submit a link to your Notebook on your Github repository. 

In [1]:
#import required libraries for the lab
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml

acquire content from postal codes page from wikipedia

In [2]:
wkpdia_pg_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wkpg_content = requests.get(wkpdia_pg_url).text
#create bs object for parsing with lxml
wkpg4_parsing = BeautifulSoup(wkpg_content,'lxml')
# capture the postcode table
wkpg_postcodes = wkpg4_parsing.find('table',class_='wikitable')

The for loop inits the dataframe, skips 'not assigned' boroughs, and renames neighborhoods to their borough's name if not assigned

In [13]:
first_iter=True
for tablerow in wkpg_postcodes.find_all('tr'):
    if first_iter:
        #init the pandas dataframe
        #first entry in table is the column names
        column_names = tablerow.text.split('\n')[1:-1]
        CanPCodes = pd.DataFrame(columns=column_names)
        first_iter = False
    elif tablerow.text.split('\n')[2]=='Not assigned':
        #ignore the row if borough is not assigned
        continue
    elif tablerow.text.split('\n')[3]=='Not assigned':
        # if neighborhood is not assigned, take on the name of borough
        modifyrow = tablerow.text.split('\n')[1:-1]
        modifyrow[-1] = modifyrow[1]
        CanPCodes = CanPCodes.append({'Postcode':modifyrow[0],'Borough':modifyrow[1],'Neighbourhood':modifyrow[2]},ignore_index=True)
    else:
        PostCode_row = tablerow.text.split('\n')[1:-1]
        CanPCodes = CanPCodes.append({'Postcode':PostCode_row[0],'Borough':PostCode_row[1],'Neighbourhood':PostCode_row[2]},ignore_index=True)

group by postcode, bourough, and aggregate neighborhoods splitting with commas
convert the resulting series to a dataframe

In [14]:
CanPCodes=CanPCodes.groupby(['Postcode','Borough'],as_index=False, sort=False).Neighbourhood.agg(', '.join)

In [15]:
CanPCodes

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [10]:
CanPCodes.shape[0]

103